[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API #Imports all the libraries needed for the dashboard to function

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

Initial data is being loaded from "dummydata.json". This data deliberately only includes the first year of the pandemic in order to demonstrate the functionality of the "Fetch Data" Widget which refreshes the API.

In [3]:
with open("dummydata.json", "rt") as INFILE:
    jsondata=json.load(INFILE) #Opens dummydata. Contains relevant information for the first year of the pandemic.


The data includes reported cases and reported deaths for the different nations that form the United Kingdom. The method of wrangling this data uses if elif statements with f strings to ensure that data that the data that meets certain conditions in the nation field are entered into the correct columns in the dataframe.

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata): #Will create DF of a dictionary
    datalist=rawdata['data']#Dumps the .json file into a dictionary
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()#Sorts dates in order
    index=pd.date_range(parse_date(dates[0]), parse_date(dates[-1]), freq='D')#Creates a range of dates to be used by the DataFrame
    NationRateDF=pd.DataFrame(index=index, columns=['ENGcases', 'ENGdeaths','SCOcases','SCOdeaths','WALcases','WALdeaths','NIcases','NIdeaths'])#Create DataFrame
    for entry in datalist:
        date=parse_date(entry['date'])
        if entry["nation"]=="England":#Since the Nation data won't be plotted on the graph, this is the solution
            for column in ['cases', 'deaths']:#These two data fields will be plotted as ENGcases and ENGdeaths
                if pd.isna(NationRateDF.loc[date, f'ENG{column}']):#Uses f strings to insert the cases and deathts data into the relevant fields  
                    value= float(entry[column]) if entry[column]!=None else 0.0 #Floats the data if it exists, otherwise enters 0.
                    NationRateDF.loc[date, f'ENG{column}']=value #Enters the data into the DF
        elif entry["nation"]=="Scotland":
            for column in ['cases', 'deaths']:
                if pd.isna(NationRateDF.loc[date, f'SCO{column}']):  
                    value= float(entry[column]) if entry[column]!=None else 0.0
                    NationRateDF.loc[date, f'SCO{column}']=value
        elif entry["nation"]=="Wales":
            for column in ['cases', 'deaths']:
                if pd.isna(NationRateDF.loc[date, f'WAL{column}']):  
                    value= float(entry[column]) if entry[column]!=None else 0.0
                    NationRateDF.loc[date, f'WAL{column}']=value
        elif entry["nation"]=="Northern Ireland":
            for column in ['cases', 'deaths']:
                if pd.isna(NationRateDF.loc[date, f'NI{column}']):  
                    value= float(entry[column]) if entry[column]!=None else 0.0
                    NationRateDF.loc[date, f'NI{column}']=value
    NationRateDF.fillna(0.0, inplace=True) #Fills in blanks with 0.
    return NationRateDF

# putting the wrangling code into a function allows it to be called again after refreshing the data through 
# the API. 
df=wrangle_data(jsondata) #Wrangles the data on launch of dashboard

## Download current data

This button will refresh the current data, then toggle the scale widget in oder to force the graph to redraw itself. It also saves the data from the API to a .JSON file titled "NationRates". This will be a new file the first time this is run.

In [1]:
def access_api():
    filt = ['areaType=nation']
    struct = {
        "nation": "areaName",
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate"}
    jsondata=Cov19API(filters=filt, structure=struct).get_json() #Retrieves the data from the API. Dumps to a new .JSON as well.
    with open("NationRates.json","wt") as OUTF:
        json.dump(jsondata, OUTF)
    return jsondata
    

In [2]:
def refresh_graph():#Forces the graph to redraw by toggling a widget
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other
    scale.value=current

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API.
    apidata=access_api()
    # wrangles the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Fetch Data',
    disabled=False,
    button_style='danger', 
    tooltip="Pulls fresh data from the API. Saves it to NationRates.json",
    icon='exclamation-triangle'
)

apibutton.on_click(api_button_callback) #On click triggers the api_button_callback function

display(apibutton)

NameError: name 'wdg' is not defined

## Graphs and Analysis

This graph represents the daily cases and deaths of each nation in the United Kingdom. Note that the graph will not display until at least one set of Nation Data is selected.

In [7]:
def nationdata_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
plots=wdg.SelectMultiple(#Allows selection of relevant data
    options=['ENGcases','ENGdeaths','SCOcases','SCOdeaths','WALcases','WALdeaths','NIcases','NIdeaths'],
    values=['ENGcases','ENGdeaths','SCOcases','SCOdeaths','WALcases','WALdeaths','NIcases','NIdeaths'],
    rows=8,
    description="Nation Data:",
    disabled=False
)
scale=wdg.RadioButtons(#Changes scale from linear to log
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)
controls=wdg.HBox([plots, scale])
graph=wdg.interactive_output(nationdata_graph, {'gcols': plots, 'gscale': scale})
display(controls, graph)

Output()

**Author and Copyright Notice** Created by Kieran Cockburn. Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).